# FIRST/LAST_VALUE

In SQL there are the clauses `FIRST_VALUE` and `LAST_VALUE`. These allow you to get the first (top) or last (bottom) value in some sort of ordered range. In this page I want to play around with these expressions to learn how they work.

The following cell contains a database definition that will be used for the experiments on this page.

In [7]:
%%bash
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name first_value_examples \
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i first_value_examples psql -U postgres -d postgres


CREATE TABLE example_table(
    col1 TEXT,
    col2 INT
);
INSERT INTO example_table(col1, col2) VALUES
('A', 10),
('B', 43),
('C', 3),
('D', 5),
('E', 32),
('F', 31);

CREATE TABLE
INSERT 0 6


The basic tables are available here.

In [38]:
%%bash
echo "=====example_table====="
docker exec first_value_examples \
    psql -U postgres -d postgres -c "SELECT * FROM example_table";

=====example_table=====
 col1 | col2 
------+------
 A    |   10
 B    |   43
 C    |    3
 D    |    5
 E    |   32
 F    |   31
(6 rows)



**Note** Don't forget to stop the container when you've finished playing with it.

In [43]:
%%bash
docker stop first_value_examples

first_value_examples


## Find out more

- <a href="https://www.geeksforgeeks.org/postgresql-first_value-function/">GeeksForGeeks `FIRST_VALUE` statement</a>;
- <a href="https://www.commandprompt.com/education/how-to-use-the-last_value-function-in-postgresql/"> Command prompt "How to Use the LAST_VALUE() Function in PostgreSQL?"</a>.

## Basic example 

### Corresponds to lowest

You can get the first value in any column from any other column using the expression `FIRST_VALUE (<value column>) OVER (ORDER BY <sort column>);`. 

So the following example shows how to use an include to query a column that contains a value in `col1` that corresponds to the lowest value in `col2`. Lowest value of `col2 = 3`, it corresponds to the record where `col1 = 'C'`. So we have got a column that contains "C" values.

**Note** values were sorted by `col2`.

In [22]:
%%bash
docker exec -i first_value_examples psql -U postgres -d postgres
SELECT 
    col1, 
    col2, 
    FIRST_VALUE (col1) OVER (ORDER BY col2)
FROM example_table;

 col1 | col2 | first_value 
------+------+-------------
 C    |    3 | C
 D    |    5 | C
 A    |   10 | C
 F    |   31 | C
 E    |   32 | C
 B    |   43 | C
(6 rows)



### Corresponds to largest

By simply adding `DESC` to the `ORDER BY` expression, you can get the value that corresponds to the largest value.

The following example duplicates the previous one, but we have `B` in the `first_value` column because it corresponds to the largest value of `col2`.

In [25]:
%%bash
docker exec -i first_value_examples psql -U postgres -d postgres
SELECT 
    col1, 
    col2, 
    FIRST_VALUE (col1) OVER (ORDER BY col2 DESC)
FROM example_table;

 col1 | col2 | first_value 
------+------+-------------
 B    |   43 | B
 E    |   32 | B
 F    |   31 | B
 A    |   10 | B
 D    |    5 | B
 C    |    3 | B
(6 rows)



### `LAST_VALUE`

Here I'm using the `LAST_VALUE` statement. It works just like `FIRST_VALUE' but selects the last value in the range.

This is almost the same as the result of the `Corresponds to largest` subsection, but note that the result is records sorted by `col2` in ascending order.

In [41]:
%%bash
docker exec -i first_value_examples psql -U postgres -d postgres
SELECT 
    col1, 
    col2, 
    LAST_VALUE (col1) OVER (
        ORDER BY col2
        RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    )
FROM example_table;

 col1 | col2 | last_value 
------+------+------------
 C    |    3 | B
 D    |    5 | B
 A    |   10 | B
 F    |   31 | B
 E    |   32 | B
 B    |   43 | B
(6 rows)

